In [1]:
import pandas as pd

In [29]:
# Load org mode table into dataframe.
df = pd.read_csv('participants.org', delimiter='|', index_col=1, skiprows=[1])
del df[df.columns[0]]
del df[df.columns[-1]]
df.index = [s.strip() for s in df.index]
df.columns = [s.strip() for s in df.columns]

df.sort_values(by='Total', ascending=False).head()

,Intermag,IOP,DPG,Total
Germany,5,,18,23
UK,4,18,,22
USA,9,,,9
France,6,,,6
Ireland,5,,,5


I found a handy set of country centroids on this site: http://gothos.info/resources/

In [34]:
country_centroids = pd.read_table('country_centroids_all.csv')
country_centroids.head()

,LAT,LONG,DMS_LAT,DMS_LONG,MGRS,JOG,DSG,AFFIL,FIPS10,SHORT_NAME,FULL_NAME,MOD_DATE,ISO3136
0,33.000000,66.0,330000,660000,42STB1970055286,NI42-09,PCLI,NaN,AF,Afghanistan,Islamic Republic of Afghanistan,2009-04-10,AF
1,41.000000,20.0,410000,200000,34TDL1589839239,NK34-08,PCLI,NaN,AL,Albania,Republic of Albania,2007-02-28,AL
2,28.000000,3.0,280000,30000,31REL0000097202,NH31-15,PCLI,NaN,AG,Algeria,People's Democratic Republic of Algeria,2011-03-03,DZ
3,-14.333333,-170.0,-142000,-1700000,1802701,NaN,PCLD,US,AS,American Samoa,Territory of American Samoa,1998-10-06,AS
4,42.500000,1.5,423000,13000,31TCH7675006383,NK31-04,PCLI,NaN,AN,Andorra,Principality of Andorra,2007-02-28,AD


In [48]:
aliases = {
    'UK': 'United Kingdom',
    'USA': 'United States',
}

def get_lat_long(country_name):
    if country_name in aliases:
        return get_lat_long(aliases[country_name])
    filtered = country_centroids[country_centroids['SHORT_NAME'] == country_name]
    try:
        row = filtered.iloc[0]
    except IndexError:
        raise KeyError(country_name)
    return row['LAT'], row['LONG']

get_lat_long('Andorra')

(42.5, 1.5)

Now to use the [Folium map library](http://python-visualization.github.io/folium/index.html).

In [51]:
import folium

In [71]:
map1 = folium.Map(location=[20, 10])
for row in df.itertuples():
    name = row.Index
    if name == 'Unknown':
        continue
    folium.CircleMarker(get_lat_long(name),
                        popup="{} ({} people)".format(name, row.Total),
                        radius=2 * (row.Total ** 0.5),
                       ).add_to(map1)
map1.zoom_start=2
map1